# Titre 
explain

Nous avons fait le choix de nous concentrer sur l'action de NVIDIA car ... 

# A. Récupération des données 

Nous utilisons Yahoo Finance (via yfinance) pour récupérer les séries boursières car c’est une source simple d’accès, largement utilisée pour des données daily OHLCV (Open, High, Low, Close, Volume), et crédible. 

L’objectif est de construire une base fiable et reproductible en combinant trois sources :
- NVDA (Yahoo Finance) : données quotidiennes sur NVIDIA (OHLCV, Adj Close, dividendes et splits). C’est la base principale du projet.
- ETFs (Yahoo Finance) : séries quotidiennes SPY / QQQ / SOXX pour capturer le contexte marché / tech / semi-conducteurs et distinguer les chocs “systématiques” des effets spécifiques à NVDA.
- FRED (pandas-datareader) : indicateurs macro-financiers VIX (régime de volatilité) et taux US 10 ans (conditions financières), utilisés comme variables de régime.

Enfin, toutes les séries sont alignées sur un calendrier unique (les dates de cotation de NVDA) afin de garantir la cohérence temporelle. Nous sauvegardons systématiquement les données en deux étapes (raw puis processed) pour assurer la reproductibilité : le projet peut être rejoué à l’identique sans dépendre d’un nouvel appel aux sources externes, ce qui fournit un backup si la disponibilité ou la structure des données venait à évoluer.

## A.1 - Importation des données 

Dans cette partie, nous construisons la base de données qui servira pour la suite du projet. L’objectif est de **fusionner trois sources** (NVDA, ETFs et indicateurs FRED) en un **dataset unique** et cohérent. Dans la section suivante, nous procéderons à un **contrôle qualité** (valeurs manquantes, doublons, cohérence temporelle) et, si nécessaire, à une **sélection des variables** pertinentes pour l’analyse et la modélisation.

On commence par importer ce qu'il faut :

In [2]:
import pandas as pd
from pandas_datareader import data as web
from pathlib import Path
import yfinance as yf 
import numpy as np

Ensuite, nous définissons les **chemins de sauvegarde** (*paths*) pour stocker des copies locales des données (*backups*), afin de garantir la **reproductibilité** et d’éviter de retélécharger systématiquement les séries. Nous fixons également les **paramètres d’extraction** : dates de début et de fin, ainsi que les tickers récupérés via Yahoo Finance.


In [3]:
#Période d’étude : 10 ans
start_date = "2015-01-01"
end_date   = "2024-12-31"

#Sélection des tickers/séries d’intérêt (ETFs + indicateurs macro) 
etf_tickers  = ["SPY", "QQQ", "SOXX"]
fred_series  = ["VIXCLS", "DGS10"]
ticker = "NVDA"

#Gestion des paths : création du dossier raw et définition des fichiers de sauvegarde
RAW_DIR = Path("data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)
nvda_raw_path = RAW_DIR / "nvda_stock_data_raw.csv"
etf_raw_path  = RAW_DIR / "etf_stock_data_raw.csv"
fred_raw_path = RAW_DIR / "fred_macro_raw.csv"

On télécharge ensuite les trois bases de données (NVDA, ETFs et séries FRED) et on les affiche systématiquement afin de vérifier rapidement la cohérence de l’output (format, variables présentes, ordre temporel).

In [4]:
## Téléchargement des données de NVIDIA

#Si le fichier raw existe déjà, on le recharge (évite de retélécharger à chaque exécution)
if nvda_raw_path.exists():
    nvda_data_2 = pd.read_csv(nvda_raw_path, parse_dates=["Date"])

#Sinon, on télécharge NVDA via Yahoo
else:
    nvda_data_2 = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        interval="1d",
        auto_adjust=False,
        actions=True,
        progress=False
    ).reset_index()

    nvda_data_2 = nvda_data_2.rename(columns={c: f"{c}_{ticker}" for c in nvda_data_2.columns if c != "Date"})
    
    #Save en cache pour la reproductibilité 
    nvda_data_2.to_csv(nvda_raw_path, index=False)

#Affihcage de contrôle de la cellue 
nvda_data_2.head()

,Date,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,NaT,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
1,2015-01-02,0.48301151394844055,0.503250002861023,0.0,0.5070000290870667,0.49524998664855957,0.503250002861023,0.0,113680000
2,2015-01-05,0.47485315799713135,0.4947499930858612,0.0,0.5047500133514404,0.4925000071525574,0.503250002861023,0.0,197952000
3,2015-01-06,0.46045637130737305,0.47975000739097595,0.0,0.4959999918937683,0.4792500138282776,0.49549999833106995,0.0,197764000
4,2015-01-07,0.4592568874359131,0.47850000858306885,0.0,0.48750001192092896,0.47699999809265137,0.4832499921321869,0.0,321808000


In [5]:
## Téléchargment de ETFs + FRED

#ETFs : (pareil que pour NVDA)
if etf_raw_path.exists():
    etf_data = pd.read_csv(etf_raw_path, parse_dates=["Date"])
else:
    etf = yf.download(
        etf_tickers,
        start=start_date,
        end=end_date,
        interval="1d",
        auto_adjust=False,
        actions=True,
        progress=False,
        group_by="column"
    )
    etf.columns = [f"{field}_{tic}" for field, tic in etf.columns]
    etf_data = etf.reset_index()
    etf_data.to_csv(etf_raw_path, index=False)

#FRED(VIXCLS, DGS10) : (pareil que pour NVDA)
if fred_raw_path.exists():
    fred_data = pd.read_csv(fred_raw_path, parse_dates=["DATE"])
else:
    fred_data = web.DataReader(fred_series, "fred", start_date, end_date).reset_index()
    fred_data.to_csv(fred_raw_path, index=False)

#Affihcages de contrôle de la cellue 
fred_data.head()
etf_data.head()

,Date,Adj Close_QQQ,Adj Close_SOXX,Adj Close_SPY,Capital Gains_QQQ,Capital Gains_SOXX,Capital Gains_SPY,Close_QQQ,Close_SOXX,Close_SPY,...,Low_SPY,Open_QQQ,Open_SOXX,Open_SPY,Stock Splits_QQQ,Stock Splits_SOXX,Stock Splits_SPY,Volume_QQQ,Volume_SOXX,Volume_SPY
0,2015-01-02,94.784447,27.559072,170.589645,0.0,0.0,0.0,102.940002,30.936666,205.429993,...,204.179993,103.760002,31.100000,206.380005,0.0,0.0,0.0,31314600,663000,121465900
1,2015-01-05,93.394058,27.039431,167.508789,0.0,0.0,0.0,101.430000,30.353333,201.720001,...,201.350006,102.489998,30.806667,204.169998,0.0,0.0,0.0,36521300,619500,169632600
2,2015-01-06,92.141792,26.436630,165.931076,0.0,0.0,0.0,100.070000,29.676666,199.820007,...,198.860001,101.580002,30.373333,202.089996,0.0,0.0,0.0,66205500,1123800,209151400
3,2015-01-07,93.329582,26.697947,167.998779,0.0,0.0,0.0,101.360001,29.969999,202.309998,...,200.880005,100.730003,29.846666,201.419998,0.0,0.0,0.0,37577400,721200,125346700
4,2015-01-08,95.115921,27.490778,170.979889,0.0,0.0,0.0,103.300003,30.860001,205.899994,...,203.990005,102.220001,30.273333,204.009995,0.0,0.0,0.0,40212600,633000,147217800


Ensuite, nous fusionnons les trois bases (NVDA, ETFs et FRED) afin d’obtenir un dataset unique, prêt pour les étapes de contrôle qualité et de préparation des variables.

In [6]:
## Création de la base finale  

#Mise à plat des noms de colonnes : yfinance peut produire un MultiIndex (ex. ("Close","NVDA"))
if isinstance(nvda_data_2.columns, pd.MultiIndex):
    nvda_data_2.columns = [
        f"{a}_{b}" if (b is not None and b != "") else str(a)
        for a, b in nvda_data_2.columns
    ]
else:
    nvda_data_2.columns = [
        "_".join(map(str, c)).strip() if isinstance(c, tuple) else str(c)
        for c in nvda_data_2.columns
    ]

#On repart d’une base NVDA “clean” (colonnes *_NVDA uniquement) pour éviter les doublons en cas de relance
nvda_cols = ["Date"] + [c for c in nvda_data_2.columns if c.endswith("_NVDA")]
nvda_base = nvda_data_2[nvda_cols].copy()


#On a remarqué que le nom de variable de la date est différent entre FRED et NVDA
#On renomme simplement cette variable pour merge : "DATE" -> "Date"
if "DATE" in fred_data.columns and "Date" not in fred_data.columns:
    fred_data = fred_data.rename(columns={"DATE": "Date"})

#Mise au bon format et tri chronologique pour le merge temporel
for df in [nvda_base, etf_data, fred_data]:
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values("Date", inplace=True)

#Indexation selon les dates de cotations de NVDA
nvda_idx = nvda_base.set_index("Date")
etf_idx  = etf_data.set_index("Date")
fred_idx = fred_data.set_index("Date")


#Alignement sur les dates NVDA (ETFs) + forward-fill pour les séries macro (FRED) en cas de jours manquants
etf_aligned  = etf_idx.reindex(nvda_idx.index)
fred_aligned = fred_idx.reindex(nvda_idx.index).ffill()

#Marge par left join 
nvda_data_2 = (
    nvda_idx
    .join(etf_aligned, how="left")
    .join(fred_aligned, how="left")
    .reset_index()
)

#Sauvegarde 
PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

processed_parquet = PROCESSED_DIR / "dataset_merged_nvda_etf_fred.parquet"
processed_csv     = PROCESSED_DIR / "dataset_merged_nvda_etf_fred.csv"

try:
    nvda_data_2.to_parquet(processed_parquet, index=False)
    print("Saved parquet:", processed_parquet)
except ImportError:
    nvda_data_2.to_csv(processed_csv, index=False)
    print("Parquet engine missing -> saved CSV instead:", processed_csv)

#Affichage de contrôle 
print("Merged nvda_data shape:", nvda_data_2.shape)
nvda_data_2.head()

Parquet engine missing -> saved CSV instead: data/processed/dataset_merged_nvda_etf_fred.csv
Merged nvda_data shape: (2516, 30)


,Date,Adj Close_QQQ,Adj Close_SOXX,Adj Close_SPY,Capital Gains_QQQ,Capital Gains_SOXX,Capital Gains_SPY,Close_QQQ,Close_SOXX,Close_SPY,...,Open_SOXX,Open_SPY,Stock Splits_QQQ,Stock Splits_SOXX,Stock Splits_SPY,Volume_QQQ,Volume_SOXX,Volume_SPY,VIXCLS,DGS10
0,2015-01-02,94.784447,27.559072,170.589645,0.0,0.0,0.0,102.940002,30.936666,205.429993,...,31.100000,206.380005,0.0,0.0,0.0,31314600.0,663000.0,121465900.0,17.79,2.12
1,2015-01-05,93.394058,27.039431,167.508789,0.0,0.0,0.0,101.430000,30.353333,201.720001,...,30.806667,204.169998,0.0,0.0,0.0,36521300.0,619500.0,169632600.0,19.92,2.04
2,2015-01-06,92.141792,26.436630,165.931076,0.0,0.0,0.0,100.070000,29.676666,199.820007,...,30.373333,202.089996,0.0,0.0,0.0,66205500.0,1123800.0,209151400.0,21.12,1.97
3,2015-01-07,93.329582,26.697947,167.998779,0.0,0.0,0.0,101.360001,29.969999,202.309998,...,29.846666,201.419998,0.0,0.0,0.0,37577400.0,721200.0,125346700.0,19.31,1.96
4,2015-01-08,95.115921,27.490778,170.979889,0.0,0.0,0.0,103.300003,30.860001,205.899994,...,30.273333,204.009995,0.0,0.0,0.0,40212600.0,633000.0,147217800.0,17.01,2.03


## A.2 - Manipulation des données

Dans cette section, nous analysons et mettons au propre la base complète `nvda_data_2`, obtenue après fusion des données NVDA, ETFs et FRED. L’objectif est de vérifier la qualité du merge (cohérence temporelle, doublons) et de préparer une base exploitable pour la suite du projet.

### A.2.1 - nvda_data_2
 Nous commençons par l’identification et le traitement des valeurs manquantes.

In [ ]:
# Tri + sécurité Date
nvda_data_2["Date"] = pd.to_datetime(nvda_data_2["Date"])
nvda_data_2 = nvda_data_2.sort_values("Date").reset_index(drop=True)

#Gestion des valeurs manquantes 
na_counts = nvda_data_2.isna().sum().sort_values(ascending=False)
print("Top colonnes avec NA:")
print(na_counts.head(15))


Top colonnes avec NA:
Date                 1
Adj Close_QQQ        1
Volume_SPY           1
Volume_SOXX          1
Volume_QQQ           1
Stock Splits_SPY     1
Stock Splits_SOXX    1
Stock Splits_QQQ     1
Open_SPY             1
Open_SOXX            1
Open_QQQ             1
Low_SPY              1
Low_SOXX             1
Low_QQQ              1
High_SPY             1
dtype: int64


**Interprétation** : Aucune valeur manquante n’est détectée. C’est cohérent avec les séries issues de Yahoo Finance, généralement complètes sur les jours de cotation. 

Nous passons ensuite à la vérification et au traitement d’éventuels doublons de dates.

In [8]:
dup_dates = nvda_data_2["Date"].duplicated().sum()
print("\nNombre de dates dupliquées:", dup_dates)

#Si doublons : on garde la dernière occurrence (par convention)
if dup_dates > 0:
    nvda_data_2 = nvda_data_2.drop_duplicates(subset=["Date"], keep="last").reset_index(drop=True)


Nombre de dates dupliquées: 0


Aucun doublon de date n’est détecté : la base est donc cohérente sur la dimension temporelle et peut être utilisée telle quelle pour la suite. 

Nous passons maintenant à une première description statistique des variables.

In [ ]:
desc_full = nvda_data_2.describe(include=[np.number]).T
desc_full.head(15)

#TODO prendre le min et max de NVDIA

,count,mean,std,min,25%,50%,75%,max
Adj Close_QQQ,2515.0,236.695524,117.789245,89.586868,133.226089,195.457916,327.252991,534.592285
Adj Close_SOXX,2515.0,95.634714,61.532319,22.689611,44.915722,72.783043,140.069466,262.575653
Adj Close_SPY,2515.0,310.206377,114.384025,154.980865,212.165367,273.141602,399.227463,598.740112
Capital Gains_QQQ,2515.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Capital Gains_SOXX,2515.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Capital Gains_SPY,2515.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Close_QQQ,2515.0,243.987634,117.048811,96.320000,141.135002,203.000000,336.635010,538.169983
Close_SOXX,2515.0,99.441692,61.481325,25.263332,48.911665,76.633331,144.755005,265.489990
Close_SPY,2515.0,334.939109,107.831576,182.860001,243.049995,300.160004,419.989990,607.809998
Dividends_QQQ,2515.0,0.006843,0.057121,0.000000,0.000000,0.000000,0.000000,0.835000


**Interprétation** : Le tableau `desc_full` fournit un premier diagnostic sur les variables numériques du dataset fusionné. 

- Les statistiques de dispersion (écart-type, minimum et maximum) mettent en évidence l’hétérogénéité des échelles (prix, volumes, indicateurs macro) et la présence possible de valeurs extrêmes, en particulier sur les volumes, ce qui est typique des séries financières. Mais on va garder ces valeurs extrêmes car il y'aurait aucune raison de les enlever. Surtout dans un context de finance de marché, où les marchés restent soumis à l'aléas avec la possibilité de mouvements "extrêmes". 

### A.2.2 - nvda_data

Dans cette partie, nous construisons une sous-base `nvda_data` contenant uniquement les variables relatives à NVIDIA. Elle sera utilisée plus tard pour mener, si besoin, une analyse centrée exclusivement sur NVDA (sans variables de contexte marché/macro).

**Remarque** : la base principale fusionnée est nommée `nvda_data_2`(et pas `nvda_data`) car le projet a été initialement développé à partir de la seule base NVDA. Lors de l’ajout ultérieur des sources externes (ETFs et FRED), nous avons conservé cette convention de nommage afin de préserver la continuité du notebook.

In [18]:
#duplication avec conservation des colonnes d'intérêt  
columns_nvda_data = ["Date", "Open_NVDA", "High_NVDA", "Low_NVDA", "Close_NVDA", "Volume_NVDA"]
nvda_data = nvda_data_2[columns_nvda_data].copy()

#sauvegarde 
OUT = Path("data/processed")
OUT.mkdir(parents=True, exist_ok=True)

nvda_data.to_csv(OUT / "nvda_data.csv", index=False)

On calcule maintenant l’asymétrie (*skewness*) de la distribution, afin de caractériser si la variable étudiée présente une dissymétrie marquée (présence plus fréquente d’extrêmes positifs ou négatifs).

In [17]:
nvda_data["ret_NVDA"] = nvda_data["Close_NVDA"].pct_change()
print("Skewness ret_NVDA (rendements):", nvda_data["ret_NVDA"].dropna().skew())

Skewness ret_NVDA (rendements): 0.6271634666393318


**Interprétation** : la *skewness* mesure l’**asymétrie** d’une distribution autour de sa moyenne. 
- Une skewness **positive** signifie que la distribution présente une **queue à droite** plus longue : on observe plus souvent des **hausses extrêmes** que des baisses extrêmes (ou, dit autrement, quelques jours de très fortes hausses tirent la distribution vers la droite). 
- Ici, `skewness = 0.627` indique une asymétrie positive **modérée** des rendements journaliers de NVDA.

**Exemple illustratif** : si la plupart des jours se situent autour de −1% à +1%, mais qu’il existe ponctuellement des journées à +6% / +10% plus fréquentes ou plus “fortes” que les journées à −6% / −10%, la skewness devient positive.

**Pourquoi calculer sur les rendements plutôt que sur les prix ?** Les prix sont non stationnaires et dominés par une tendance de long terme, ce qui rend leur distribution peu interprétable. Les **rendements** (variations relatives) sont l’objet standard en finance : ils permettent de comparer les mouvements dans le temps et entre actifs, et décrivent directement le risque (variabilité, asymétrie, extrêmes).

Fin de la récupération et du traitement des données. 